# Lending Club Case Study

In [259]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,6))
import warnings
warnings.filterwarnings('ignore')

<Figure size 1500x600 with 0 Axes>

## Load Data

In [260]:
loans_df = pd.read_csv("data\loan.csv")
loans_df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,92187,92174,2500,2500,1075.0,36 months,8.07%,78.42,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39713,90665,90607,8500,8500,875.0,36 months,10.28%,275.38,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39714,90395,90390,5000,5000,1325.0,36 months,8.07%,156.84,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39715,90376,89243,5000,5000,650.0,36 months,7.43%,155.38,A,A2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Studying the Data

In [261]:
loans_df = loans_df[['application_type', #Indicates whether the loan is an individual application or a joint application with two co-borrowers - does having joint result worser loans ?
          'int_rate', # Interest Rate on the loan - Does Higher Interest rates causes more defaults
          'loan_amnt', # The listed amount of the loan applied for by the borrower - Does the loan amount result in defaulting
          'issue_d', #The month which the loan was funded
          'total_pymnt', # Payments received to date for total amount funded
          'total_rec_int', # total received interest
          'total_rec_prncp', # total received principal
          'total_rec_late_fee', # total received late fee
          'recoveries', # post charge off gross recovery
          'loan_status', # is it 'Fully paid', 'Current', 'Charged-off' (defaulted)
          'verification_status', # Indicates if income was verified by LC - Should check if this has more incidents of defaulting
          'term', # Does a longer 'term' have an effect on the defaulting of loans
          'grade', # Are higher grade loans more riskier because of interest rates ?
          'delinq_amnt', # The past-due amount owed for the accounts on which the borrower is now delinquent.
          'emp_length' , # Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years
          'purpose', # A category provided by the borrower for the loan request.
          'annual_inc', # The self-reported annual income provided by the borrower during registration.
          'annual_inc_joint', #The combined self-reported annual income provided by the co-borrowers during registration
          'total_bc_limit', # Total bankcard high credit/credit limit
          'avg_cur_bal', # Average current balance of all accounts
          'revol_util', # Your revolving utilization rate compares your credit debt to your total credit limit, Experian recommends that you keep your revolving utilization rate below 30%. The lower your rate, the better for loans  ?
          'addr_state' # The state provided by the borrower in the loan application
          ]]


## Cleaning the Data

In [262]:
# Check shape of dataset
total_rows = loans_df.shape[0]
print("# Rows: {} \n# Columns: {}".format(total_rows, loans_df.shape[1]))

# Check how many INDIVIDUAL Accounts
individual_rows = loans_df[loans_df['application_type'] == 'INDIVIDUAL'].shape[0]
print("All records have application_type INDIVIDUAL: {}".format(individual_rows == total_rows))
loans_df.drop(loans_df.annual_inc_joint.name, axis=1, inplace=True)


# Rows: 39717 
# Columns: 22
All records have application_type INDIVIDUAL: True


In [263]:
# find unique values for total_bc_limit avg_cur_bal
bc_limit_rows = loans_df[loans_df['total_bc_limit'].isna()].shape[0]
print("All records have bc_limit_rows NaN: {}".format(bc_limit_rows == total_rows))
# remove total_bc_limit
loans_df.drop(loans_df.total_bc_limit.name, axis=1, inplace=True)
loans_df

All records have bc_limit_rows NaN: True


,application_type,int_rate,loan_amnt,issue_d,total_pymnt,total_rec_int,total_rec_prncp,total_rec_late_fee,recoveries,loan_status,verification_status,term,grade,delinq_amnt,emp_length,purpose,annual_inc,avg_cur_bal,revol_util,addr_state
0,INDIVIDUAL,10.65%,5000,Dec-11,5863.155187,863.16,5000.00,0.00,0.00,Fully Paid,Verified,36 months,B,0,10+ years,credit_card,24000.0,NaN,83.70%,AZ
1,INDIVIDUAL,15.27%,2500,Dec-11,1008.710000,435.17,456.46,0.00,117.08,Charged Off,Source Verified,60 months,C,0,< 1 year,car,30000.0,NaN,9.40%,GA
2,INDIVIDUAL,15.96%,2400,Dec-11,3005.666844,605.67,2400.00,0.00,0.00,Fully Paid,Not Verified,36 months,C,0,10+ years,small_business,12252.0,NaN,98.50%,IL
3,INDIVIDUAL,13.49%,10000,Dec-11,12231.890000,2214.92,10000.00,16.97,0.00,Fully Paid,Source Verified,36 months,C,0,10+ years,other,49200.0,NaN,21%,CA
4,INDIVIDUAL,12.69%,3000,Dec-11,3513.330000,1037.39,2475.94,0.00,0.00,Current,Source Verified,60 months,B,0,1 year,other,80000.0,NaN,53.90%,OR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,INDIVIDUAL,8.07%,2500,Jul-07,2822.969293,322.97,2500.00,0.00,0.00,Fully Paid,Not Verified,36 months,A,0,4 years,home_improvement,110000.0,NaN,13.10%,CO
39713,INDIVIDUAL,10.28%,8500,Jul-07,9913.491822,1413.49,8500.00,0.00,0.00,Fully Paid,Not Verified,36 months,C,0,3 years,credit_card,18000.0,NaN,26.90%,NC
39714,INDIVIDUAL,8.07%,5000,Jul-07,5272.161128,272.16,5000.00,0.00,0.00,Fully Paid,Not Verified,36 months,A,0,< 1 year,debt_consolidation,100000.0,NaN,19.40%,MA
39715,INDIVIDUAL,7.43%,5000,Jul-07,5174.198551,174.20,5000.00,0.00,0.00,Fully Paid,Not Verified,36 months,A,0,< 1 year,other,200000.0,NaN,0.70%,MD


In [264]:
# find unique values for total_bc_limit avg_cur_bal
avg_cur_bal_rows = loans_df[loans_df['avg_cur_bal'].isna()].shape[0]
print("All records have avg_cur_bal NaN: {}".format(avg_cur_bal_rows == total_rows))
# remove total_bc_limit
loans_df.drop(loans_df.avg_cur_bal.name, axis=1, inplace=True)
loans_df

All records have avg_cur_bal NaN: True


,application_type,int_rate,loan_amnt,issue_d,total_pymnt,total_rec_int,total_rec_prncp,total_rec_late_fee,recoveries,loan_status,verification_status,term,grade,delinq_amnt,emp_length,purpose,annual_inc,revol_util,addr_state
0,INDIVIDUAL,10.65%,5000,Dec-11,5863.155187,863.16,5000.00,0.00,0.00,Fully Paid,Verified,36 months,B,0,10+ years,credit_card,24000.0,83.70%,AZ
1,INDIVIDUAL,15.27%,2500,Dec-11,1008.710000,435.17,456.46,0.00,117.08,Charged Off,Source Verified,60 months,C,0,< 1 year,car,30000.0,9.40%,GA
2,INDIVIDUAL,15.96%,2400,Dec-11,3005.666844,605.67,2400.00,0.00,0.00,Fully Paid,Not Verified,36 months,C,0,10+ years,small_business,12252.0,98.50%,IL
3,INDIVIDUAL,13.49%,10000,Dec-11,12231.890000,2214.92,10000.00,16.97,0.00,Fully Paid,Source Verified,36 months,C,0,10+ years,other,49200.0,21%,CA
4,INDIVIDUAL,12.69%,3000,Dec-11,3513.330000,1037.39,2475.94,0.00,0.00,Current,Source Verified,60 months,B,0,1 year,other,80000.0,53.90%,OR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,INDIVIDUAL,8.07%,2500,Jul-07,2822.969293,322.97,2500.00,0.00,0.00,Fully Paid,Not Verified,36 months,A,0,4 years,home_improvement,110000.0,13.10%,CO
39713,INDIVIDUAL,10.28%,8500,Jul-07,9913.491822,1413.49,8500.00,0.00,0.00,Fully Paid,Not Verified,36 months,C,0,3 years,credit_card,18000.0,26.90%,NC
39714,INDIVIDUAL,8.07%,5000,Jul-07,5272.161128,272.16,5000.00,0.00,0.00,Fully Paid,Not Verified,36 months,A,0,< 1 year,debt_consolidation,100000.0,19.40%,MA
39715,INDIVIDUAL,7.43%,5000,Jul-07,5174.198551,174.20,5000.00,0.00,0.00,Fully Paid,Not Verified,36 months,A,0,< 1 year,other,200000.0,0.70%,MD


In [265]:
# Clean columns datatype
loans_df.info()
# int_rate remove %, precision 2, float64
# issue_d create year column and month column
# emp_length convert to int64
# revol_util remove %, precision 2, float64

loans_df['int_rate'] = loans_df.int_rate.apply(lambda x: float(str(x).replace("%","")))
loans_df.info()
loans_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   application_type     39717 non-null  object 
 1   int_rate             39717 non-null  object 
 2   loan_amnt            39717 non-null  int64  
 3   issue_d              39717 non-null  object 
 4   total_pymnt          39717 non-null  float64
 5   total_rec_int        39717 non-null  float64
 6   total_rec_prncp      39717 non-null  float64
 7   total_rec_late_fee   39717 non-null  float64
 8   recoveries           39717 non-null  float64
 9   loan_status          39717 non-null  object 
 10  verification_status  39717 non-null  object 
 11  term                 39717 non-null  object 
 12  grade                39717 non-null  object 
 13  delinq_amnt          39717 non-null  int64  
 14  emp_length           38642 non-null  object 
 15  purpose              39717 non-null 

,application_type,int_rate,loan_amnt,issue_d,total_pymnt,total_rec_int,total_rec_prncp,total_rec_late_fee,recoveries,loan_status,verification_status,term,grade,delinq_amnt,emp_length,purpose,annual_inc,revol_util,addr_state
0,INDIVIDUAL,10.65,5000,Dec-11,5863.155187,863.16,5000.00,0.00,0.00,Fully Paid,Verified,36 months,B,0,10+ years,credit_card,24000.0,83.70%,AZ
1,INDIVIDUAL,15.27,2500,Dec-11,1008.710000,435.17,456.46,0.00,117.08,Charged Off,Source Verified,60 months,C,0,< 1 year,car,30000.0,9.40%,GA
2,INDIVIDUAL,15.96,2400,Dec-11,3005.666844,605.67,2400.00,0.00,0.00,Fully Paid,Not Verified,36 months,C,0,10+ years,small_business,12252.0,98.50%,IL
3,INDIVIDUAL,13.49,10000,Dec-11,12231.890000,2214.92,10000.00,16.97,0.00,Fully Paid,Source Verified,36 months,C,0,10+ years,other,49200.0,21%,CA
4,INDIVIDUAL,12.69,3000,Dec-11,3513.330000,1037.39,2475.94,0.00,0.00,Current,Source Verified,60 months,B,0,1 year,other,80000.0,53.90%,OR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,INDIVIDUAL,8.07,2500,Jul-07,2822.969293,322.97,2500.00,0.00,0.00,Fully Paid,Not Verified,36 months,A,0,4 years,home_improvement,110000.0,13.10%,CO
39713,INDIVIDUAL,10.28,8500,Jul-07,9913.491822,1413.49,8500.00,0.00,0.00,Fully Paid,Not Verified,36 months,C,0,3 years,credit_card,18000.0,26.90%,NC
39714,INDIVIDUAL,8.07,5000,Jul-07,5272.161128,272.16,5000.00,0.00,0.00,Fully Paid,Not Verified,36 months,A,0,< 1 year,debt_consolidation,100000.0,19.40%,MA
39715,INDIVIDUAL,7.43,5000,Jul-07,5174.198551,174.20,5000.00,0.00,0.00,Fully Paid,Not Verified,36 months,A,0,< 1 year,other,200000.0,0.70%,MD


In [266]:
loans_df['issue_year'] = pd.DatetimeIndex(pd.to_datetime(loans_df.issue_d,format='%b-%y')).year
loans_df['issue_month'] = pd.DatetimeIndex(pd.to_datetime(loans_df.issue_d,format='%b-%y')).month
loans_df

,application_type,int_rate,loan_amnt,issue_d,total_pymnt,total_rec_int,total_rec_prncp,total_rec_late_fee,recoveries,loan_status,...,term,grade,delinq_amnt,emp_length,purpose,annual_inc,revol_util,addr_state,issue_year,issue_month
0,INDIVIDUAL,10.65,5000,Dec-11,5863.155187,863.16,5000.00,0.00,0.00,Fully Paid,...,36 months,B,0,10+ years,credit_card,24000.0,83.70%,AZ,2011,12
1,INDIVIDUAL,15.27,2500,Dec-11,1008.710000,435.17,456.46,0.00,117.08,Charged Off,...,60 months,C,0,< 1 year,car,30000.0,9.40%,GA,2011,12
2,INDIVIDUAL,15.96,2400,Dec-11,3005.666844,605.67,2400.00,0.00,0.00,Fully Paid,...,36 months,C,0,10+ years,small_business,12252.0,98.50%,IL,2011,12
3,INDIVIDUAL,13.49,10000,Dec-11,12231.890000,2214.92,10000.00,16.97,0.00,Fully Paid,...,36 months,C,0,10+ years,other,49200.0,21%,CA,2011,12
4,INDIVIDUAL,12.69,3000,Dec-11,3513.330000,1037.39,2475.94,0.00,0.00,Current,...,60 months,B,0,1 year,other,80000.0,53.90%,OR,2011,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,INDIVIDUAL,8.07,2500,Jul-07,2822.969293,322.97,2500.00,0.00,0.00,Fully Paid,...,36 months,A,0,4 years,home_improvement,110000.0,13.10%,CO,2007,7
39713,INDIVIDUAL,10.28,8500,Jul-07,9913.491822,1413.49,8500.00,0.00,0.00,Fully Paid,...,36 months,C,0,3 years,credit_card,18000.0,26.90%,NC,2007,7
39714,INDIVIDUAL,8.07,5000,Jul-07,5272.161128,272.16,5000.00,0.00,0.00,Fully Paid,...,36 months,A,0,< 1 year,debt_consolidation,100000.0,19.40%,MA,2007,7
39715,INDIVIDUAL,7.43,5000,Jul-07,5174.198551,174.20,5000.00,0.00,0.00,Fully Paid,...,36 months,A,0,< 1 year,other,200000.0,0.70%,MD,2007,7


In [267]:
# emp_length convert to int64
def formatEmp_length(data):
    return str(data).replace("+ years","").replace(" year","").replace("< ","").replace("s","").replace("nan","-1")

loans_df['emp_length'] = loans_df.emp_length.apply(lambda x: formatEmp_length(x))
loans_df['emp_length'] = loans_df['emp_length'].astype(int)


In [268]:
# revol_util remove %, precision 2, float64
loans_df['revol_util'] = loans_df.revol_util.apply(lambda x: float(str(x).replace("%","")))
loans_df.revol_util.value_counts()

0.00     977
0.20      63
63.00     62
40.70     58
66.70     58
        ... 
25.74      1
47.36      1
24.65      1
10.61      1
7.28       1
Name: revol_util, Length: 1089, dtype: int64

In [269]:
# total rows with one null value
print("total rows with at least one null value: {}".format(loans_df.isnull().any(axis = 1).sum()))
#loans_df[loans_df.isnull().any(axis=1)]
loans_df['revol_util'] = loans_df.revol_util.apply(lambda x: -1 if pd.isna(x) else x  )
loans_df[loans_df.isnull().any(axis=1)]
print("total rows with at least one null value: {}".format(loans_df.isnull().any(axis = 1).sum()))


total rows with at least one null value: 50
total rows with at least one null value: 0


In [270]:
loans_df

,application_type,int_rate,loan_amnt,issue_d,total_pymnt,total_rec_int,total_rec_prncp,total_rec_late_fee,recoveries,loan_status,...,term,grade,delinq_amnt,emp_length,purpose,annual_inc,revol_util,addr_state,issue_year,issue_month
0,INDIVIDUAL,10.65,5000,Dec-11,5863.155187,863.16,5000.00,0.00,0.00,Fully Paid,...,36 months,B,0,10,credit_card,24000.0,83.7,AZ,2011,12
1,INDIVIDUAL,15.27,2500,Dec-11,1008.710000,435.17,456.46,0.00,117.08,Charged Off,...,60 months,C,0,1,car,30000.0,9.4,GA,2011,12
2,INDIVIDUAL,15.96,2400,Dec-11,3005.666844,605.67,2400.00,0.00,0.00,Fully Paid,...,36 months,C,0,10,small_business,12252.0,98.5,IL,2011,12
3,INDIVIDUAL,13.49,10000,Dec-11,12231.890000,2214.92,10000.00,16.97,0.00,Fully Paid,...,36 months,C,0,10,other,49200.0,21.0,CA,2011,12
4,INDIVIDUAL,12.69,3000,Dec-11,3513.330000,1037.39,2475.94,0.00,0.00,Current,...,60 months,B,0,1,other,80000.0,53.9,OR,2011,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,INDIVIDUAL,8.07,2500,Jul-07,2822.969293,322.97,2500.00,0.00,0.00,Fully Paid,...,36 months,A,0,4,home_improvement,110000.0,13.1,CO,2007,7
39713,INDIVIDUAL,10.28,8500,Jul-07,9913.491822,1413.49,8500.00,0.00,0.00,Fully Paid,...,36 months,C,0,3,credit_card,18000.0,26.9,NC,2007,7
39714,INDIVIDUAL,8.07,5000,Jul-07,5272.161128,272.16,5000.00,0.00,0.00,Fully Paid,...,36 months,A,0,1,debt_consolidation,100000.0,19.4,MA,2007,7
39715,INDIVIDUAL,7.43,5000,Jul-07,5174.198551,174.20,5000.00,0.00,0.00,Fully Paid,...,36 months,A,0,1,other,200000.0,0.7,MD,2007,7
